This script pulls all Olympus snapshot proposal votes, divided into individual files per snapshot

In [2]:
from datetime import datetime
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [3]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [4]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
)

In [5]:
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=10000,
  where=[
    snapshot.Proposal.space == 'olympusdao.eth',
    snapshot.Proposal.state == 'closed',
  ]
)

In [6]:
proposals_snapshots = sg.query_df([
  proposals.title,
  proposals.id,
  proposals.body
])

In [7]:
proposals_choices = sg.query(proposals.choices)

In [8]:
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_a', 'option_b', 'option_c', 'option_d', 'option_e','option_f','option_g'])

In [9]:
olympus_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [10]:
##let's view the output just to make sure
olympus_governance_view

,proposals_title,proposals_id,proposals_body,option_a,option_b,option_c,option_d,option_e,option_f,option_g
0,OIP-94: Interim Ranged Stability Policy Levers,0x6413984c8f1076e3f46f196810ba51781f25f4d2f1eb...,abipup\n \n 2 days ago\n\nSumma...,Inverse bonds & adjust liquidity,Do nothing,None,None,None,None,None
1,OIP-93: Mint and Sync,0xf0e62d0a9eeb47a295bab9a8d0cc84c0d4de0a8cb000...,Summary:\n\nEnable minting into liquidity pool...,"1. Implement, no change to RR","2. Implement, change RR",3. Do not implement,None,None,None,None
2,TAP-12 Pilot Deposit on Tribe Turbo,0x7d191f4a7f2ea3d138c07f73d568b93c96fc5ffce267...,Summary\n\nSeek authority from the community t...,Approve Pilot Deposit,Do not approve pilot deposit,None,None,None,None,None
3,TAP-11 Ribbon Finance Whitelist,0xe52a4af6307bea40a34f2eaf9a93f11c8abf382020cc...,Summary: Seek authority from the community to ...,Allow Ribbon Finance,Do not allow Ribbon Finance,None,None,None,None,None
4,OIP - 92 : Whitelist FEI as a treasury Reserve...,0xe9aa47f39bc5ace6ad5a19c08007a3683386e19c1390...,Summary\n\nWhitelist FEI as our next reserve a...,Approve FEI in the Treasury,Do not approve FEI in Treasury,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
192,OIP-5: Lower LP Staking Rewards,QmTrwGU99RMqTeeCKy5oW9Le9M2mXHHdNeEWzn7T67watz,Summary: We should decrease LP staking rewards...,For: Option 1,For: Option 2,Against,None,None,None,None
193,Deposit SLP,QmZhSNWAzrDy5KiGSeBsKBXUHW6D4vJnfCuqmdai8LPETd,Summary: The DAO currently holds liquidity tha...,Deposit; keep Onsen,Deposit; no Onsen,Don't deposit,None,None,None,None
194,Loosen Policy,QmZJb2evEjsxtLz92XBuhApWty69fVqUpFHzuVjMjCd3Se,Summary: Let's adjust system parameters away f...,Do it,Don't do it,None,None,None,None,None
195,Add DAI Bonds,QmUC5xa4YaWZBV7fy241svBQ8bHSMksm8kB7avk1WyWv9m,Summary: We can launch bonds using DAI instead...,Add; remove sales,Add; phase out sales,Add; keep sales,Do not add,None,None,None


In [11]:
total_proposals = len(olympus_governance_view)
total_proposals

197

We now have the number of snapshots, which we can use as the max number of times we will fetch a votes report

In [12]:
print(olympus_governance_view.iloc[,1])

SyntaxError: invalid syntax (885679355.py, line 1)

In [13]:
x = 0
while x <197:
    print(olympus_governance_view.iloc[x,1])
    proposal_id = olympus_governance_view.iloc[x,1]

    vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=10000,
    where=[
      snapshot.Vote.proposal == proposal_id
    ]
    )
    voting_snapshots = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice
])
    root_path = '/Users/ianbrillembourg/Downloads/OIPS/'
    path = root_path+proposal_id+'autosnap.csv'
    voting_snapshots.to_csv(path, index = False)
    ##voting_snapshots.to_csv(r'/Users/ianbrillembourg/Downloads/OIPS/Otestsnapshot.csv', index = False)
    breakpoint()
    x=x+1

0x6413984c8f1076e3f46f196810ba51781f25f4d2f1ebee606455929cbbc7d7c8
0xf0e62d0a9eeb47a295bab9a8d0cc84c0d4de0a8cb000a38fb28e301cd96b4eb6
0x7d191f4a7f2ea3d138c07f73d568b93c96fc5ffce2674271fbced88406d26fa7
0xe52a4af6307bea40a34f2eaf9a93f11c8abf382020cca2d370ffaa0337620b6b
0xe9aa47f39bc5ace6ad5a19c08007a3683386e19c139011c136fa0007f0592289
0xf90119baf2b7bca681d41ec611e093363dcff38ff5efa1c3aada55da91aac3a5
0x675a4acc6c9402b2269ea61a425c73b185ede2640a7aca11a1ba052eb5a2c050
0xbbc9ee3b8a3a7b8ba97fa2c5014810a2a82151e79d468044271fdb782c40ed86
0x807d75e51065c6d28651bf77627425cdac5835bd192c181a5186efe1d4042096
0x70babccd32c31752f60275e0b2045b1bb73c7a98606fed54c0b54f6c2f7e1e62
0x01dfb4809fd8c986b13a9c04919a75ee31bf47beb2e05b7fa09b8019f580c059
0xceb3f950f22310b0245599ca97b3c3d25ba3cbda74183744c60a0b03cbdd165b
0xc7839708a47c3d09aab4d9eee91e5e5757c202a447e5586b5d3fbb9a9dc1cf49
0x3529ecff8159a24307c720662ede59675feb7e99ea6b35b86a1aab58babfdbe9
0x73ec03590c5d5bf39405bc1638bf9c7e01b4777ac4bd687450a76fc7b565